### 의존 파이썬 패키지

In [1]:
!pip install crc16 redis-py-cluster

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [10]:
import random
from rediscluster import RedisCluster
import warnings
import time
import crc16

warnings.filterwarnings("ignore")

In [11]:
redis_host = "redis-master.redis-farm.svc.cluster.local"

In [12]:
import rediscluster
  
startup_nodes = [{ "host": redis_host, "port": "6379" }]
client = rediscluster.RedisCluster(startup_nodes=[dict(host=redis_host, port="6379")], decode_responses=True, skip_full_coverage_check=True)

# Retrieve the slot mapping (equivalent of "CLUSTER SLOTS")
print("This is the cluster slots information")
print(client.cluster('slots'))

# Calculate hash slot for the key 'foo' and TechTrip
print("Check the slot numbers of two data")
print(client.cluster('keyslot', 'foo'))
print(client.cluster('keyslot', 'TechTrip'))

# We can set a key without regard for which shard it will be stored on
# as the Redis client SDK will manage this mapping for us.
print("Set Redis data as the Key is foo, its value is bar")
client.set('foo', 'bar')
print(client.get('foo'))

RedisClusterException: ERROR sending 'cluster slots' command to redis server: {'host': 'redis-master.redis-farm.svc.cluster.local', 'port': '6379'}

In [10]:
def now():
    return int(time.time() * 1000)

In [ ]:
class InputData:
    def __init__(self, ci, latitude, longitude):
        self.ci = ci
        self.latitude = latitude
        self.longitude = longitude
        

class RedisClient:
    
    def __init__(self, redis_host, geo_ttl=30):
        self.r = RedisCluster(startup_nodes=[{"host": redis_host, "port": "6379"}], decode_responses=True,
                                          skip_full_coverage_check=True)
        self.p = r.pipeline()
        
        # ttl 단위 minute
        self.geo_ttl = geo_ttl
        
        # hashtags 값 미리 구하기 (샤드 개수에 따라 달라짐)
        nodes = self.r.cluster_nodes()
        masters = [node for node in nodes if "master" in node["flags"]]
        slots_to_master_id = dict()

        for m in masters:
            for slot in m["slots"]:
                slots_to_master_id[slot] = m["id"]

        temp_nodes = set()
        self.hashtags = []
        for i in range(16384):
            node_id = slots_to_master_id[
                r.cluster_keyslot("{" + str(i) + "}_last_loc")
            ]
            if node_id not in temp_nodes:
                temp_nodes.add(node_id)
                self.hashtags.append(i)
            if len(temp_nodes) == len(masters):
                break

        self.hashtags_count = len(self.hashtags)

        print("hashtags : ", self.hashtags)
        
    
        
    # insert multi
    def insert_multi_data(self, input_data_list):
        for input_data in input_data_list:
            ts = now()
            hashtag = self.hashtags[
                (crc16.crc16xmodem(bytes(input_data.ci, "utf-8")) % 16384) % self.hashtags_count
            ]

            # geoadd
            geo_key = "{" + str(hashtag) + "}" + "_last_loc"
            self.p.geoadd(geo_key, input_data.longitude, input_data.latitude, input_data.ci)

            # location series with timestamp
            loc_seq_key = f"location_seq:{input_data.ci}"
            self.p.zadd(loc_seq_key, {f"{input_data.latitude}:{input_data.longitude}:{ts}": ts})

            # ttl 분이 지난 과거 위치는 삭제
            self.p.zremrangebyscore(loc_seq_key, 0, ts - self.geo_ttl*60*1000)

        self.p.execute()



    # 현재 위치
    def current_location(self, ci):
        seq_last_ts = self.r.zrange(f"location_seq:{ci}", -1, -1)
        latitude, longitude, ts = seq_last_ts[0].split(":")
        return float(latitude), float(longitude), int(ts)



    # georadius
    def georadiusbymember(self, ci, radius, unit="km"):
        latitude, longitude, ts = self.current_location(ci)
        cis = []
        for k in self.hashtags:
            key = "{" + str(k) + "}" + "_last_loc"
            cis += [
                x for x in self.r.georadius(key, longitude, latitude, radius, unit=unit, withdist=True, withcoord=True)
                if x[0] != ci
            ]

        return cis

In [ ]:
redis_client = RedisClient(redis_host, geo_ttl=30)

RedisClusterException: ERROR sending 'cluster slots' command to redis server: {'host': 'redis-master.redis-farm.svc.cluster.local', 'port': '6379'}

### Insert data

In [ ]:
input_datas = [
    InputData(
        ci='user_1',
        latitude=37.01757,
        longitude=128.522931
    ),
    InputData(
        ci='user_2',
        latitude=37.01755,
        longitude=128.522831
    ),
    
]

redis_client.insert_multi_data(input_datas)

### 최신 위치

In [ ]:
redis_client.current_location(input_datas[0].ci)

### georadius

In [ ]:
redis_client.georadiusbymember(input_datas[0].ci, radius=0.1, unit="km")

In [ ]:
# 모든 key 삭제

# redis_client.r.flushall()